In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `whisper train` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate 

Train-test dataset

In [ ]:
import os
from datasets import Dataset, Audio, DatasetDict
import pandas as pd
import re
import librosa

# Define the folder path
folder_path = "./"  # Replace with the actual folder path

# Define the words and their corresponding patterns
words = {
    "sora open browser": r"[bB]rowser \((\d+)\)\.mp3",
    "sora open chrome": r"[cC]hrome \((\d+)\)\.mp3",
    "sora end class": r"[eE]nd [cC]lass \((\d+)\)\.mp3",
    "sora open google": r"[gG]oogle \((\d+)\)\.mp3",
    "sora open kahoot": r"[kK]ahoot \((\d+)\)\.mp3",
    "sora open mentimeter": r"[mM]entimeter \((\d+)\)\.mp3",
    "sora": r"[sS]ora \((\d+)\)\.mp3",
    "sora start class": r"[sS]tart [cC]lass \((\d+)\)\.mp3",
}

# Create a list of audio file paths and sentences
audio_paths = []
sentences = []

for filename in os.listdir(folder_path):
    for word, pattern in words.items():
        match = re.match(pattern, filename)
        if match:
            file_path = os.path.join(folder_path, filename)
            audio_paths.append(file_path)
            sentences.append(word)
            break  # Move to the next filename if a match is found

# Create the dataset
data = {
    "audio": audio_paths,
    "sentence": sentences,
}
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)
dataset = dataset.cast_column("audio", Audio())

# Split the dataset into train and test sets
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# Create a DatasetDict
dataset = DatasetDict({
    'train': dataset['train'],
    'test': dataset['test']
})

print(dataset)  # To see the output, run the code.

# Push the dataset to the Hugging Face Hub
dataset.push_to_hub("EdwardFang09/IEE4912_Dataset_v2")

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 40
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10
    })
})


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/EdwardFang09/IEE4912_Dataset_v2/commit/21eccac113c334064aac05181552fc7aa68e67cd', commit_message='Upload dataset', commit_description='', oid='21eccac113c334064aac05181552fc7aa68e67cd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/EdwardFang09/IEE4912_Dataset_v2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='EdwardFang09/IEE4912_Dataset_v2'), pr_revision=None, pr_num=None)

Benchmark dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("EdwardFang09/IEE4912_Dataset_v2")

README.md:   0%|          | 0.00/523 [00:00<?, ?B/s]

benchmark-00000-of-00001.parquet:   0%|          | 0.00/173k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating benchmark split:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
import os
from datasets import Dataset, Audio, DatasetDict
import pandas as pd
import re
import librosa

# Define the folder path
folder_path = "./"  # Replace with the actual folder path

# Define the words and their corresponding patterns
words = {
    "sora open chrome": r"chrome.mp3",
    "sora open kahoot": r"kahoot.mp3",
    "sora open mentimeter": r"mentimeter.mp3",
}

# Create a list of audio file paths and sentences
audio_paths = []
sentences = []

for filename in os.listdir(folder_path):
    for word, pattern in words.items():
        match = re.match(pattern, filename)
        if match:
            file_path = os.path.join(folder_path, filename)
            audio_paths.append(file_path)
            sentences.append(word)
            break  # Move to the next filename if a match is found

# Create the benchmark dataset
data = {
    "audio": audio_paths,
    "sentence": sentences,
}
df = pd.DataFrame(data)
benchmark_dataset = Dataset.from_pandas(df)
benchmark_dataset = benchmark_dataset.cast_column("audio", Audio())

In [ ]:
dataset["benchmark"] = benchmark_dataset

In [ ]:
dataset.push_to_hub("EdwardFang09/IEE4912_Dataset_v2")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/EdwardFang09/IEE4912_Dataset_v2/commit/a2d72f69b44f15bb977466cea09c6844607aa276', commit_message='Upload dataset', commit_description='', oid='a2d72f69b44f15bb977466cea09c6844607aa276', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/EdwardFang09/IEE4912_Dataset_v2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='EdwardFang09/IEE4912_Dataset_v2'), pr_revision=None, pr_num=None)